GoogleDriveをマウント

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


読み順推定

In [3]:
import glob
import os
import xml.etree.ElementTree as ET
import math
from scipy.stats import rankdata
import shutil

dir = 'drive/MyDrive' #保存先フォルダを指定
os.makedirs(f'{dir}/推定データ(コマ順位ベース)', exist_ok = True)


c = 0
xmldir = 'drive/MyDrive/annotations' #"annotation"フォルダのあるフォルダを指定
file = glob.glob(f'{xmldir}/*.xml')


for b in range (len(file)):
    file_A = file[b]
    filename = os.path.split(file_A)[1]
    manga = filename[:-4]
    tree = ET.parse(f'{xmldir}/{filename}')
    root = tree.getroot()


    for book in root.iter('book'):
        bookName = book.attrib['title']

    widths,heights = [],[]

    p_ids = []
    p_xmins,p_xmaxs,p_ymins,p_ymaxs = [],[],[],[]
    p_xcs,p_ycs=[],[]
    p_txts=[]

    p = 0
    for page in root.iter('page'):

        widths.append(page.attrib['width'])
        heights.append(page.attrib['height'])

        ids = []
        xmins,ymins,xmaxs,ymaxs = [],[],[],[]
        xcs,ycs = [],[]#各テキストの中心座標を格納するリスト
        txts=[]
        for text in page.iter('text'):
            id =  text.attrib['id']
            xmin, xmax, ymin, ymax, txt = int(text.attrib['xmin']), int(text.attrib['xmax']), int(text.attrib['ymin']), int(text.attrib['ymax']), text.text
            xc, yc = (xmin+xmax)/2, (ymin+ymax)/2 #テキストの中心座標を計算
            ids.append(id)
            xmins.append(xmin)
            ymins.append(ymin)
            xmaxs.append(xmax)
            ymaxs.append(ymax)
            xcs.append(xc)
            ycs.append(yc)
            txts.append(txt)

        p_ids.append(ids)
        p_xmins.append(xmins)
        p_ymins.append(ymins)
        p_xmaxs.append(xmaxs)
        p_ymaxs.append(ymaxs)
        p_xcs.append(xcs)
        p_ycs.append(ycs)
        p_txts.append(txts)
        p += 1

    frames,p_fids,p_fxmins,p_fymins,p_fxmaxs,p_fymaxs = [],[],[],[],[],[]
    p_fxcs,p_fycs=[],[]
    p_fdists = []

    p=0

    for page in root.iter('page'):
        f = 0
        fids, fxmins, fymins, fxmaxs, fymaxs = [],[],[],[],[]
        fxcs, fycs = [], []
        fdists = []
        for frame in page.iter('frame'):
            fid, fxmin, fymin, fxmax, fymax = frame.attrib['id'], int(frame.attrib['xmin']), int(frame.attrib['ymin']), int(frame.attrib['xmax']), int(frame.attrib['ymax'])
            fxc, fyc = (fxmin+fxmax)/2, (fymin+fymax)/2 #フレームの中心座標を計算
            #右上からの距離を計算
            if  fxc > int(page.attrib['width'])/2: #ページ2分割：右
                fdist = math.sqrt((int(page.attrib['width']) - fxc)**2 + fyc**2)
            else: #ページ2分割：左
                fdist = math.sqrt((int(page.attrib['width'])/2 - fxc)**2 + fyc**2) + int(page.attrib['width'])
            fids.append(fid)
            fxmins.append(fxmin)
            fymins.append(fymin)
            fxmaxs.append(fxmax)
            fymaxs.append(fymax)
            fxcs.append(fxc)
            fycs.append(fyc)
            fdists.append(fdist)
            f += 1

        frames.append(f)
        p_fids.append(fids)
        p_fxmins.append(fxmins)
        p_fymins.append(fymins)
        p_fxmaxs.append(fxmaxs)
        p_fymaxs.append(fymaxs)
        p_fxcs.append(fxcs)
        p_fycs.append(fycs)
        p_fdists.append(fdists)
        p += 1


    #テキストの所属コマ判定
    p_txtsinframe = []
    err = 0

    for i in range(p):
        txtsinframe = []
        for j in range(len(p_txts[i])):
            cornerinframe = [] #フレーム内に含まれるテキストの頂点の数
            centerinframe = []
            distfromframe = [] #各テキストのフレームからの距離(重複があった場合に使用)
            for k in range(frames[i]):
                hit = 0 #テキストの中心がコマの範囲内か
                count = 0 #四隅のうちコマの範囲内にある頂点の数
                if p_fxmins[i][k] < p_xcs[i][j] < p_fxmaxs[i][k] and p_fymins[i][k] < p_ycs[i][j] < p_fymaxs[i][k]:
                    hit +=1
                if p_fxmins[i][k] < p_xmins[i][j] < p_fxmaxs[i][k] and p_fymins[i][k] < p_ymins[i][j] < p_fymaxs[i][k]:
                    count +=1
                if p_fxmins[i][k] < p_xmins[i][j] < p_fxmaxs[i][k] and p_fymins[i][k] < p_ymaxs[i][j] < p_fymaxs[i][k]:
                    count +=1
                if p_fxmins[i][k] < p_xmaxs[i][j] < p_fxmaxs[i][k] and p_fymins[i][k] < p_ymins[i][j] < p_fymaxs[i][k]:
                    count +=1
                if p_fxmins[i][k] < p_xmaxs[i][j] < p_fxmaxs[i][k] and p_fymins[i][k] < p_ymaxs[i][j] < p_fymaxs[i][k]:
                    count +=1
                cornerinframe.append(count)
                centerinframe.append(hit)
                dist = math.sqrt((p_xcs[i][j]-p_fxcs[i][k])**2+(p_ycs[i][j]-p_fycs[i][k])**2)
                distfromframe.append(dist)

            if centerinframe.count(1) > 1 : #テキストの中心が2つ以上のコマの範囲内にある場合
                indexes = [i for i, x in enumerate(centerinframe) if x == 1]
                d_dists = []
                for l in range(len(indexes)):
                    #中心の距離と、テキスト・コマのそれぞれ四隅の距離を足し合わせる
                    d_dist = math.sqrt(abs(p_xcs[i][j]-p_fxcs[i][indexes[l]])**2+abs(p_ycs[i][j]-p_fycs[i][indexes[l]])**2)*\
                    (math.sqrt(abs(p_xmins[i][j]-p_fxmins[i][indexes[l]])**2+abs(p_ymins[i][j]-p_fymins[i][indexes[l]])**2)+\
                    math.sqrt(abs(p_xmins[i][j]-p_fxmins[i][indexes[l]])**2+abs(p_ymaxs[i][j]-p_fymaxs[i][indexes[l]])**2)+\
                    math.sqrt(abs(p_xmaxs[i][j]-p_fxmaxs[i][indexes[l]])**2+abs(p_ymins[i][j]-p_fymins[i][indexes[l]])**2)+\
                    math.sqrt(abs(p_xmaxs[i][j]-p_fxmaxs[i][indexes[l]])**2+abs(p_ymaxs[i][j]-p_fymaxs[i][indexes[l]])**2))
                    d_dists.append(d_dist)
                near = min(d_dists) #距離が小さいほうのコマに属する
                n = d_dists.index(near)
                txtsinframe.append(indexes[n])
            elif centerinframe.count(1) == 1: #テキストの中心が1つのコマの範囲内にのみある場合
                txtsinframe.append(centerinframe.index(1))
            else: #テキストがコマの範囲外にある場合
                n = 4
                while not n in cornerinframe: 
                    if n < 0:
                        break
                    n -= 1
                if n < 0:
                    err += 1
                    break
                if cornerinframe.count(n) > 1 :
                    indexes = [i for i, x in enumerate(cornerinframe) if x == n]
                    d_dists = []
                    for l in range(len(indexes)):
                        #中心の距離と、テキスト・コマのそれぞれ四隅の距離を足し合わせる
                        d_dist = math.sqrt(abs(p_xcs[i][j]-p_fxcs[i][indexes[l]])**2+abs(p_ycs[i][j]-p_fycs[i][indexes[l]])**2)*\
                        (math.sqrt(abs(p_xmins[i][j]-p_fxmins[i][indexes[l]])**2+abs(p_ymins[i][j]-p_fymins[i][indexes[l]])**2)+\
                        math.sqrt(abs(p_xmins[i][j]-p_fxmins[i][indexes[l]])**2+abs(p_ymaxs[i][j]-p_fymaxs[i][indexes[l]])**2)+\
                        math.sqrt(abs(p_xmaxs[i][j]-p_fxmaxs[i][indexes[l]])**2+abs(p_ymins[i][j]-p_fymins[i][indexes[l]])**2)+\
                        math.sqrt(abs(p_xmaxs[i][j]-p_fxmaxs[i][indexes[l]])**2+abs(p_ymaxs[i][j]-p_fymaxs[i][indexes[l]])**2))
                        d_dists.append(d_dist)
                    near = min(d_dists) #距離が小さいほうのコマに属する
                    n = d_dists.index(near)
                    txtsinframe.append(indexes[n])
                else :
                    txtsinframe.append(cornerinframe.index(n))
        p_txtsinframe.append(txtsinframe)
    



    b_dists = []
    b_order = []
    b_t_ids,b_t_xmins,b_t_ymins,b_t_xmaxs,b_t_ymaxs = [],[],[],[],[]
    b_txts = []
    for i in range(p):
        p_dists = []
        p_order = []
        p_ftxts = []
        p_t_ids,p_t_xmins,p_t_ymins,p_t_xmaxs,p_t_ymaxs=[],[],[],[],[]
        for j in range(frames[i]):
            dists = []
            ftxts = []
            t_ids,t_xmins,t_ymins,t_xmaxs,t_ymaxs=[],[],[],[],[]
            for k in range(len(p_txtsinframe[i])):
                if p_txtsinframe[i][k] == j:
                    dist = math.sqrt((p_fxmaxs[i][j] - p_xcs[i][k])**2 + (p_fymins[i][j] - p_ycs[i][k])**2)
                    dists.append(dist)
                    ftxts.append(p_txts[i][k])
                    t_ids.append(p_ids[i][k])
                    t_xmins.append(p_xmins[i][k])
                    t_ymins.append(p_ymins[i][k])
                    t_xmaxs.append(p_xmaxs[i][k])
                    t_ymaxs.append(p_ymaxs[i][k])
            order = rankdata(dists)
            order = order.astype(int)-1
            p_dists.append(dists)
            p_order.append(order.tolist())
            p_ftxts.append(ftxts)
            p_t_ids.append(t_ids)
            p_t_xmins.append(t_xmins)
            p_t_ymins.append(t_ymins)
            p_t_xmaxs.append(t_xmaxs)
            p_t_ymaxs.append(t_ymaxs)

        b_dists.append(p_dists)
        b_order.append(p_order)
        b_txts.append(p_ftxts)
        b_t_ids.append(p_t_ids)
        b_t_xmins.append(p_t_xmins)
        b_t_ymins.append(p_t_ymins)
        b_t_xmaxs.append(p_t_xmaxs)
        b_t_ymaxs.append(p_t_ymaxs)



    #コマの順位付け
    p_f_order = []
    for i in range(len(p_fdists)):
        f_order = rankdata(p_fdists[i])
        f_order = f_order.astype(int)-1
        f_order = f_order.tolist()
        dup = [x for x in set(f_order) if f_order.count(x) > 1]
        if len(dup)>0:
            for j in range(len(dup)):
                dists = []
                indexes = []
                for k in range(len(f_order)):
                    if f_order[k] == dup[j]:
                        indexes.append(k)
                        dists.append(math.sqrt((int(page.attrib['width']) - p_fxmaxs[i][k])**2 + p_fymins[i][k]))
                order = rankdata(dists)
                order = order.astype(int)-1
                order = order.tolist()
                for l in range(len(order)):
                    f_order[indexes[l]] += order[l]
        p_f_order.append(f_order)
    
    #入れ子式のリストを作成
    book_order = []
    book_fid = []
    book_tid = []

    for i in range(p):
        page_order = []
        page_fid = []
        page_tid = []
        count = 0
        for j in range(frames[i]):
            frame_order = []
            x = p_f_order[i].index(j)
            fid =p_fids[i][x]
            page_fid.append(fid)
            count = 0
            for k in range(len(p_txtsinframe[i])):
                if p_txtsinframe[i][k] == p_f_order[i].index(j):
                    tid = b_t_ids[i][x][b_order[i][x].index(count)]
                    page_tid.append(tid)
                    frame_order.append(tid)
                    count += 1
            page_order.append(frame_order)
        book_fid.append(page_fid)
        book_tid.append(page_tid)
        book_order.append(page_order)


    #xmlに書き出し
    with open(f'{manga}_o.xml', 'w') as f:
        print('<?xml version="1.0" encoding="UTF-8" ?>',file = f)
        print(f'<book title="{manga}">', file = f)
        print('<pages>', file = f)
        for i in range(p):
            if len(p_fids[i]) > 0:
                print(f'<page index="{i}" width="{widths[i]}" height="{heights[i]}">', file = f)
                print(f'<frames>', file = f)
            else:
                print(f'<page index="{i}" width="{widths[i]}" height="{heights[i]}"/>', file = f)
            count = 0
            for j in range(frames[i]):
                x = p_f_order[i].index(j)
                if len(b_txts[i][p_f_order[i].index(j)]) > 0:
                    print(f'<frame id="{p_fids[i][x]}" xmin="{p_fxmins[i][x]}" ymin="{p_fymins[i][x]}" xmax="{p_fxmaxs[i][x]}" ymax="{p_fymaxs[i][x]}">', file = f)
                else:
                    print(f'<frame id="{p_fids[i][x]}" xmin="{p_fxmins[i][x]}" ymin="{p_fymins[i][x]}" xmax="{p_fxmaxs[i][x]}" ymax="{p_fymaxs[i][x]}"/>', file = f)
                count = 0
                for k in range(len(p_txtsinframe[i])):
                    if p_txtsinframe[i][k] == p_f_order[i].index(j):
                        print(f'<text id="{b_t_ids[i][x][b_order[i][x].index(count)]}" xmin="{b_t_xmins[i][x][b_order[i][x].index(count)]}" ymin="{b_t_ymins[i][x][b_order[i][x].index(count)]}" xmax="{b_t_xmaxs[i][x][b_order[i][x].index(count)]}" ymax="{b_t_ymaxs[i][x][b_order[i][x].index(count)]}" >{b_txts[i][x][b_order[i][x].index(count)]}</text>', file = f)
                        count += 1
                if len(b_txts[i][p_f_order[i].index(j)]) > 0:
                    print('</frame>', file = f)
            if len(p_fids[i]) > 0:
                print(f'</frames>', file = f)
            if len(p_fids[i]) > 0:
                print('</page>', file = f)
        print('</pages>', file = f)
        print('</book>', file = f)
        f.close()

        shutil.move(f'{manga}_o.xml', f'{dir}/推定データ(コマ順位ベース)/{manga}_o.xml')


    #jsonに書き出し
    os.makedirs(f'{dir}/推定データ(コマ順位ベース)/{manga}', exist_ok = True)
    for i in range(len(book_order)):
        if i < 10:
            fileTitle = (f'{bookName}_00{i}.json')
        elif i < 100:
            fileTitle = (f'{bookName}_0{i}.json')
        else:
            fileTitle = (f'{bookName}_{i}.json')
        with open(f'{fileTitle}', 'w') as f:
            print('{', file = f)
            print(f'  "title": "{bookName}",', file = f)
            print(f'  "index": "{i}",', file = f)
            count = 0
            if book_fid[i] == 0:
                print('  "framesOrder": []', file = f)
            else:
                print('  "framesOrder": [', file = f)
                for j in range(len(book_fid[i])):
                    if j < len(book_fid[i])-1:
                        print(f'    "{book_fid[i][j]}",', file = f)
                    else:
                        print(f'    "{book_fid[i][j]}"', file = f)
                print('  ],', file = f)
            if book_tid[i] == 0:
                print('  "textOrder": []', file = f)
            else:
                print('  "textOrder": [', file = f)
                for j in range(len(book_tid[i])):
                    if j < len(book_tid[i])-1:
                        print(f'    "{book_tid[i][j]}",', file = f)
                    else:
                        print(f'    "{book_tid[i][j]}"', file = f)
                print('  ],', file = f)
            print('  "alignment": {', file = f)
            print('    "connected": [', file = f)
            for k in range(frames[i]):
                print('      {', file = f)
                print(f'        "frameId": "{book_fid[i][k]}",', file = f)
                if len(book_order[i][k]) == 0:
                    print(f'        "textsInFrame": []', file = f)    
                else:
                    print(f'        "textsInFrame": [', file = f)
                    for l in range(len(book_order[i][k])):
                        if l < len(book_order[i][k])-1:
                            print(f'          "{book_order[i][k][l]}",', file = f)
                        else:
                            print(f'          "{book_order[i][k][l]}"', file = f)
                    print('        ]', file = f)
                if k < frames[i]-1:
                    print('      },', file = f)
                else:
                    print('      }', file = f)
            print('    ]', file = f)
            print('  }', file = f)
            print('}', file = f)
            f.close()

            shutil.move(f'{fileTitle}', f'{dir}/推定データ(コマ順位ベース)/{manga}/{fileTitle}')

    print(b+1, '/', len(file), 'completed.')
    c += 1

print(f'{c}/{len(file)}個のファイルの処理に成功しました。')

KeyboardInterrupt: ignored